In [7]:
import pandas as pd, glob, json, os

rows = []
for cfg in ["SINGLE","MULTI"]:
    for f in glob.glob(f"raw_results/{cfg}/HumanEval_*.json"):
        d = json.load(open(f))
        rows.append({"task": os.path.basename(f)[:-5],
                     "config": cfg,
                     "passed": d["passed"],
                     "tokens": d["tokens"]})

df = pd.DataFrame(rows)
pivot = df.pivot(index="task", columns="config", values="passed")
print("MULTI가 추가로 통과한 문제 수:",
      ((pivot["MULTI"]==True) & (pivot["SINGLE"]==False)).sum())


KeyError: 'passed'